#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.dep_parser import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 指定语料库路径
data_file = '../data/raw/test_data.tsv'

# 读取 TSV 格式语料
# 数据存储于 DataFrame 对象
data = load_data(data_file)
print(f'成功读取 {len(data)} 条平行句对')

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print('数据前 5 行如下：')
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型中文依存句法标注：UD 标注集**

In [3]:
# === 加载模型：LLM UD 标注集 ===

# 语种：汉语
lang = 'chinese'

# 标注集：UD
# https://universaldependencies.org/u/dep/
# https://universaldependencies.org/treebanks/zh_gsd/index.html
tagset = 'ud'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | kimi-k2.5 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 思考模式：关闭
enable_thinking = False

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_zh_ud_parser = DEPParser(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
    enable_thinking=enable_thinking,
)
t1 = time.time()
print('LLM UD 中文依存句法标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM UD 中文依存句法标注模型加载完毕！
加载耗时：1.93s


In [4]:
# === 标注原文：LLM UD 标注集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_zh_ud_annos = annotate_data(
    data['source'],
    llm_zh_ud_parser,
)

# 打印标注结果
print("\n=== LLM UD 中文依存句法标注结果预览 ===")
display_anno(llm_zh_ud_annos[0])

Dependency Parsing: 100%|████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1465.67it/s]


=== LLM UD 中文依存句法标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', '海滨', 'compound'), ('近', '海滨', 'amod'), ('海滨', '路', 'nmod'), ('的', '海滨', 'mark:rel'), ('一', '条', 'nummod'), ('条', '路', 'clf'), ('大', '路', 'amod'), ('路', '上', 'nmod'), ('上', '押', 'obl'), ('，', '押', 'punct'), ('一', '队', 'nummod'), ('队', '清兵', 'clf'), ('清兵', '押', 'nsubj'), ('手', '执', 'obl'), ('执', '清兵', 'acl'), ('刀', '枪', 'conj'), ('枪', '执', 'obj'), ('，', '押', 'punct'), ('押', 'ROOT', 'root'), ('着', '押', 'aux'), ('七', '辆', 'nummod'), ('辆', '囚车', 'clf'), ('囚车', '押', 'obj'), ('，', '押', 'punct'), ('冲', '风', 'compound:vv'), ('风', '冒', 'conj'), ('冒', '寒', 'compound:vv'), ('寒', '行', 'advcl'), ('，', '行', 'punct'), ('向', '北', 'case'), ('北', '行', 'obl'), ('而', '行', 'mark'), ('行', '押', 'advcl'), ('。', '押', 'punct')]


#### **4. 中文依存句法标注模型对比：LLM & Stanza**

In [5]:
# === 加载模型：Stanza UD 标注集 ===

# 语种：汉语
lang = 'chinese'

# 标注集：UD
# https://universaldependencies.org/u/dep/
# https://universaldependencies.org/treebanks/zh_gsd/index.html
tagset = 'ud'

# 标注模式：Stanza 本地模型
mode = 'local'

# Stanza UD 模型：gsdsimp_charlm
# https://stanfordnlp.github.io/stanza/depparse.html
# 首次加载需从网络将模型下载至本地
t0 = time.time()
stanza_zh_ud_parser = DEPParser(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('Stanza UD 中文依存句法标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

2026-02-14 12:08:37 INFO: "zh" is an alias for "zh-hans"
2026-02-14 12:08:37 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package          |
--------------------------------
| tokenize  | gsdsimp          |
| pos       | gsdsimp_charlm   |
| lemma     | gsdsimp_nocharlm |
| depparse  | gsdsimp_charlm   |

2026-02-14 12:08:37 INFO: Using device: cuda
2026-02-14 12:08:37 INFO: Loading: tokenize
2026-02-14 12:08:37 INFO: Loading: pos
2026-02-14 12:08:39 INFO: Loading: lemma
2026-02-14 12:08:40 INFO: Loading: depparse
2026-02-14 12:08:40 INFO: Done loading processors!


Stanza UD 中文依存句法标注模型加载完毕！
加载耗时：11.76s


In [6]:
# === 标注原文：Stanza UD 标注集 ===

stanza_zh_ud_annos = annotate_data(
    data['source'],
    stanza_zh_ud_parser,
)

# 打印标注结果
print("\n=== Stanza UD 中文依存句法标注结果预览 ===")
display_anno(stanza_zh_ud_annos[0])

Dependency Parsing: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.29it/s]


=== Stanza UD 中文依存句法标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', '海滨', 'nmod'), ('近', '海滨', 'case'), ('海滨', '路上', 'nmod'), ('的', '海滨', 'case'), ('一', '路上', 'nummod'), ('条', '一', 'clf'), ('大', '路上', 'case'), ('路上', '行', 'nmod:tmod'), ('，', '路上', 'punct'), ('一', '队', 'nummod'), ('队', '手', 'nmod'), ('清兵', '手', 'compound'), ('手', '行', 'nsubj'), ('执', '行', 'advcl'), ('刀枪', '执', 'obj'), ('，', '执', 'punct'), ('押', '行', 'advcl'), ('着', '押', 'aux'), ('七', '囚车', 'nummod'), ('辆', '七', 'clf'), ('囚车', '押', 'obj'), ('，', '押', 'punct'), ('冲', '风', 'compound'), ('风', '冒寒', 'nsubj'), ('冒寒', '行', 'advcl'), ('，', '冒寒', 'punct'), ('向', '北', 'case'), ('北', '行', 'obl'), ('而', '行', 'mark'), ('行', 'ROOT', 'root'), ('。', '行', 'punct')]


In [7]:
# === 模型对比：LLM UD & Stanza UD ===

print('中文依存句法标注模型对比（UD 标注集）：LLM & Stanza')
compare_annos(
    llm_zh_ud_annos,
    stanza_zh_ud_annos,
    annos_1_name="LLM",
    annos_2_name="Stanza",
)

中文依存句法标注模型对比（UD 标注集）：LLM & Stanza

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
Jaccard: 0.122
LLM: {('一', '条', 'nummod'), ('枪', '执', 'obj'), ('清兵', '押', 'nsubj'), ('上', '押', 'obl'), ('近', '海滨', 'amod'), ('条', '路', 'clf'), ('大', '路', 'amod'), ('寒', '行', 'advcl'), ('行', '押', 'advcl'), ('队', '清兵', 'clf'), ('海滨', '路', 'nmod'), ('执', '清兵', 'acl'), ('辆', '囚车', 'clf'), ('的', '海滨', 'mark:rel'), ('七', '辆', 'nummod'), ('手', '执', 'obl'), ('江南', '海滨', 'compound'), ('押', 'ROOT', 'root'), ('风', '冒', 'conj'), ('冲', '风', 'compound:vv'), ('刀', '枪', 'conj'), ('冒', '寒', 'compound:vv'), ('路', '上', 'nmod')}
Stanza: {('近', '海滨', 'case'), ('大', '路上', 'case'), ('路上', '行', 'nmod:tmod'), ('条', '一', 'clf'), ('刀枪', '执', 'obj'), ('押', '行', 'advcl'), ('一', '路上', 'nummod'), ('七', '囚车', 'nummod'), ('执', '行', 'advcl'), ('辆', '七', 'clf'), ('江南', '海滨', 'nmod'), ('海滨', '路上', 'nmod'), ('风', '冒寒', 'nsubj'), ('冒寒', '行', 'advcl'), ('队', '